# • Introduction

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Hierarchical Series

In many applications, a set of time series is hierarchically organized. Examples include the presence of geographic levels, products, or categories that define different types of aggregations. 

In such scenarios, forecasters are often required to provide predictions for all disaggregate and aggregate series. A natural desire is for those predictions to be **"coherent"**, that is, for the bottom series to add up precisely to the forecasts of the aggregated series.

\begin{align}
        y_{\mathrm{Total},\tau} = y_{\beta_{1},\tau}+y_{\beta_{2},\tau}+y_{\beta_{3},\tau}+y_{\beta_{4},\tau} 
        \qquad \qquad \qquad \qquad \qquad \\
        \mathbf{y}_{[a],\tau}=\left[y_{\mathrm{Total},\tau},\; y_{\beta_{1},\tau}+y_{\beta_{2},\tau},\;y_{\beta_{3},\tau}+y_{\beta_{4},\tau}\right]^{\intercal} 
        \qquad
        \mathbf{y}_{[b],\tau}=\left[ y_{\beta_{1},\tau},\; y_{\beta_{2},\tau},\; y_{\beta_{3},\tau},\; y_{\beta_{4},\tau} \right]^{\intercal}
\end{align}

![Figure 1. A two level time series hierarchical structure, with four bottom level variables.](./imgs/hierarchical_motivation1.png)

Figure 1. shows a simple hierarchical structure where we have four bottom-level series, two middle-level series, and the top level representing the total aggregation. The aggregations or coherency constraints of the structure are:

\begin{align}
\mathbf{S}_{[a,b][b]}
=
\begin{bmatrix}
\mathbf{A}_{\mathrm{[a][b]}} \\ 
           \\
           \\
\mathbf{I}_{\mathrm{[b][b]}} \\
           \\
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 & 1 & 1 \\
1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1 \\
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}
\end{align}

where $\mathbf{A}_{[a,b][b]}$ aggregates the bottom series to the upper levels, and $\mathbf{I}_{\mathrm{[b][b]}}$ is an identity matrix. The representation of the hierarchical series is then:

\begin{align}
\mathbf{y}_{[a,b],\tau} = \mathbf{S}_{[a,b][b]} \mathbf{y}_{[b],\tau}
\end{align}


To visualize an example, in Figure 2. One can think of the hierarchical time series structure levels to represent different geographical aggregations. For example, in Figure 2. the top level is the total aggregation of series within a country, the middle level being its states and the bottom level its regions.

![Figure 2. A hierarchy can be composed of geographic levels. In this example the top level corresponds to country aggregation, middle level to states, and bottom level to regions.](./imgs/hierarchical_motivation2.png)

## 2. Hierarchical Forecast

To achieve **"coherency"**, most statistical solutions to the hierarchical forecasting challenge implement a two-stage reconciliation process.  
1. First, we obtain a set of the base forecast $\mathbf{\hat{y}}_{[a,b],\tau}$ 
2. Later, we reconcile them into coherent forecasts $\mathbf{\tilde{y}}_{[a,b],\tau}$.

Most hierarchical reconciliation methods can be expressed by the following transformations:

\begin{align}
\tilde{\mathbf{y}}_{[a,b],\tau} = \mathbf{S}_{[a,b][b]} \mathbf{P}_{[b][a,b]} \hat{\mathbf{y}}_{[a,b],\tau}
\end{align}

The HierarchicalForecast library offers a Python collection of reconciliation methods, datasets, evaluation and visualization tools for the task. Among its available reconciliation methods we have `BottomUp`, `TopDown`, `MiddleOut`, `MinTrace`, `ERM`. Among its probabilistic coherent methods we have `Normality`, `Bootstrap`, `PERMBU`.

## 3. Example

In [ ]:
%%capture
!pip install hierarchicalforecast
!pip install -U numba statsforecast datasetsforecast

### Wrangling Data

In [1]:
import numpy as np
import pandas as pd

# compute base forecast no coherent
from statsforecast.models import Naive
from statsforecast.core import StatsForecast

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.methods import BottomUp, TopDown
from hierarchicalforecast.core import HierarchicalReconciliation

In [3]:
# Create Figure 1. synthetic data
ds = np.arange(1,9)
r1 = ds * (10**1)
r2 = ds * (10**1)
r3 = ds * (10**2)
r4 = ds * (10**2)

ys = np.concatenate([r1, r2, r3, r4])
ds = np.tile(ds, 4)
unique_ids = ['r1'] * 8 + ['r2'] * 8 + ['r3'] * 8 + ['r4'] * 8
top_level = 'Australia'
middle_level = ['State1'] * 16 + ['State2'] * 16
bottom_level = unique_ids

bottom_df = dict(unique_id=unique_ids, ds=ds, y=ys,
                 top_level=top_level, 
                 middle_level=middle_level, 
                 bottom_level=bottom_level)
bottom_df = pd.DataFrame(bottom_df)
bottom_df.groupby('unique_id').head(2)

unique_id  ds    y  top_level middle_level bottom_level
0         r1   1   10  Australia       State1           r1
1         r1   2   20  Australia       State1           r1
8         r2   1   10  Australia       State1           r2
9         r2   2   20  Australia       State1           r2
16        r3   1  100  Australia       State2           r3
17        r3   2  200  Australia       State2           r3
24        r4   1  100  Australia       State2           r4
25        r4   2  200  Australia       State2           r4

In [14]:
# Create hierarchical structure and constraints
hierarchy_levels = [['top_level'],
                    ['top_level', 'middle_level'],
                    ['top_level', 'middle_level', 'bottom_level']]
Y_hier_df, S_df, tags = aggregate(df=bottom_df, spec=hierarchy_levels)
Y_hier_df = Y_hier_df.reset_index()
print('S_df.shape', S_df.shape)
print('Y_hier_df.shape', Y_hier_df.shape)

S_df.shape (7, 4)
Y_hier_df.shape (56, 3)


In [15]:
Y_hier_df.groupby('unique_id').head(2)

unique_id  ds    y
0             Australia   1  220
1             Australia   2  440
8      Australia/State1   1   20
9      Australia/State1   2   40
16     Australia/State2   1  200
17     Australia/State2   2  400
24  Australia/State1/r1   1   10
25  Australia/State1/r1   2   20
32  Australia/State1/r2   1   10
33  Australia/State1/r2   2   20
40  Australia/State2/r3   1  100
41  Australia/State2/r3   2  200
48  Australia/State2/r4   1  100
49  Australia/State2/r4   2  200

In [16]:
S_df

Australia/State1/r1  ...  Australia/State2/r4
Australia                            1.0  ...                  1.0
Australia/State1                     1.0  ...                  0.0
Australia/State2                     0.0  ...                  1.0
Australia/State1/r1                  1.0  ...                  0.0
Australia/State1/r2                  0.0  ...                  0.0
Australia/State2/r3                  0.0  ...                  0.0
Australia/State2/r4                  0.0  ...                  1.0

[7 rows x 4 columns]

### Base Predictions

In [17]:
# Split train/test sets
Y_test_df  = Y_hier_df.groupby('unique_id').tail(4)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

# Compute base auto-ETS predictions
# Careful identifying correct data freq, this data quarterly 'Q'
fcst = StatsForecast(df=Y_train_df,
                     models=[Naive()],
                     freq='Q', n_jobs=-1)
Y_hat_df = fcst.forecast(h=4, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

### Reconciliation

In [18]:
# You can select a reconciler from our collection
reconcilers = [BottomUp()] # MinTrace(method='mint_shrink')
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, 
                          Y_df=Y_fitted_df,
                          S=S_df, tags=tags)
Y_rec_df.groupby('unique_id').head(2)

ds  Naive  Naive/BottomUp
unique_id                                     
Australia             5  880.0           880.0
Australia             6  880.0           880.0
Australia/State1      5   80.0            80.0
Australia/State1      6   80.0            80.0
Australia/State1/r1   5   40.0            40.0
Australia/State1/r1   6   40.0            40.0
Australia/State1/r2   5   40.0            40.0
Australia/State1/r2   6   40.0            40.0
Australia/State2      5  800.0           800.0
Australia/State2      6  800.0           800.0
Australia/State2/r3   5  400.0           400.0
Australia/State2/r3   6  400.0           400.0
Australia/State2/r4   5  400.0           400.0
Australia/State2/r4   6  400.0           400.0

## References

- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)<br>
- [Orcutt, G.H., Watts, H.W., & Edwards, J.B.(1968). Data aggregation and information loss. The American 
Economic Review, 58 , 773{787).](http://www.jstor.org/stable/1815532)<br>
- [Disaggregation methods to expedite product line forecasting. Journal of Forecasting, 9 , 233–254. 
doi:10.1002/for.3980090304.](https://onlinelibrary.wiley.com/doi/abs/10.1002/for.3980090304)<br>
- [Wickramasuriya, S. L., Athanasopoulos, G., & Hyndman, R. J. (2019). \"Optimal forecast reconciliation for
hierarchical and grouped time series through trace minimization\". Journal of the American Statistical Association, 
114 , 804–819. doi:10.1080/01621459.2018.1448825.](https://robjhyndman.com/publications/mint/)<br>
- [Ben Taieb, S., & Koo, B. (2019). Regularized regression for hierarchical forecasting without 
unbiasedness conditions. In Proceedings of the 25th ACM SIGKDD International Conference on Knowledge 
Discovery & Data Mining KDD '19 (p. 1337{1347). New York, NY, USA: Association for Computing Machinery.](https://doi.org/10.1145/3292500.3330976)<br>